In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path

from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from typing import List, Optional
from pydantic import BaseModel, Field


In [2]:
from dotenv import load_dotenv

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "travel_planner_v1.0"

In [4]:
landmark_prices = pd.read_csv('data/egypt.csv')
luxor_landmark = pd.read_csv('data/temp/places.csv')
luxor_restaurants = pd.read_csv('data/temp/restaurants.csv')
luxor_art = pd.read_csv('data/temp/artgallery.csv')

In [5]:
all_documents = []
for _, row in landmark_prices.iterrows():
    text = f"""
    Governorate: {row['Governorate/City']}
    Site: {row['Place']}
    Egyptian Ticket: {row['Egyptian']}
    Egyptian Student Ticket: {row['EgyptianStudent']}
    Foreign Ticket: {row['Foreign']}
    Foreign Student Ticket: {row['ForeignStudent']}
    Visiting Times: {row['VisitingTimes']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'landmark_prices'}))


In [6]:

for _, row in luxor_landmark.iterrows():
    text = f"""
    Site: {row['title']}
    Description: {row['description']}
    Site Category Name: {row['categoryName']}
    Site Categories: {row['Categories']}
    Location: {row['address']}
    Star Rating: {row['totalScore']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'Luxor Landmarks'}))

    





In [7]:
for _, row in luxor_restaurants.iterrows():
    text = f"""
    Name: {row['title']}
    Restaurant Category Name: {row['categoryName']}
    Restaurant Categories: {row['Categories']}
    Address: {row['address']}
    Price Range: {row['price']}
    Star Rating: {row['totalScore']}
    Website: {row['website']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'Luxor Restaurants'}))

In [8]:
for _, row in luxor_art.iterrows():
    text = f"""
    Art Gallery Name: {row['title']}
    Art Gallery Categories: {row['Categories']}
    Art Gallery Address: {row['address']}
    Art Gallery Website: {row['website']}
    Star Rating: {row['totalScore']}
    """
    all_documents.append(Document(page_content=text, metadata={"source": 'Luxor Art Galleries'}))

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")




c:\Users\mosai\anaconda3\envs\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
path = Path('faiss_mpnetv2_v0')
if not path.exists():
    vectorstore = FAISS.from_documents(all_documents, embeddings)
    vectorstore.save_local('faiss_mpnetv2_v0')




In [11]:
vectorstore = FAISS.load_local("faiss_mpnetv2_v0", embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

In [31]:
llm_model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq", temperature=0)
# llm_model = init_chat_model('deepseek-r1-distill-llama-70b', model_provider='groq', temperature=0)

In [13]:
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm_model,
#     chain_type="stuff",        # "stuff" means it will stuff all retrieved docs into the prompt
#     retriever=retriever
# )

# Structured Output

In [ ]:
class Activity(BaseModel):
    time: str = Field(..., description="Time of the activity")
    activity: str = Field(..., description="Name of the activity")
    location: str = Field(..., description="Location name")
    price_range: Optional[str] = Field(None, description="Price range or cost")

class DayPlan(BaseModel):
    day: str = Field(..., description="Theme of the Day or Day label, e.g., 'Day 1'")
    activities: List[Activity] = Field(..., description="Activities planned for the day")
    approximate_cost: str = Field(..., description="Total cost for the day")

class TravelItinerary(BaseModel):
    days: List[DayPlan] = Field(..., description="List of days with planned activities")
    total_approximate_cost: str = Field(..., description="Total cost for the trip")
    notes: Optional[str] = Field(None, description="Any additional notes or assumptions")

In [15]:
# prompt_template = PromptTemplate(
#     input_variables=["context", "user_query", "favorite_places", "visitor_type", "num_days", "budget"],
#     template="""You are a helpful travel planner AI.
# Use the context below, which contains information about ticket prices, place descriptions, restaurant details, and art gallery information.

# Context:
# {context}

# User Query:
# {user_query}

# Additional Preferences:
# - Favorite types of places: {favorite_places}
# - Visitor type: {visitor_type} (e.g., Egyptian, Egyptian student, Foreign, or foreign student)
# - Number of travel days: {num_days}
# - Overall budget for all days: {budget} EGP
# - Exclude hotels from the plan.
# - Ensure that the itinerary includes at least 3 meals per day.

# Based on the above, return a detailed itinerary in a JSON format. 
# The JSON should have the following structure:

# {
#   "days": [
#     {
#       "day": "Day 1",
#       "activities": [
#          {
#            "time": "8:00 am",
#            "activity": "Breakfast",
#            "location": "Bab Sharq Luxor",
#            "price_range": "E£100-200"
#          },
#          {
#            "time": "9:00 am",
#            "activity": "Visit",
#            "location": "Luxor Museum",
#            "ticket_cost": "400 EGP"
#          },
#          ... // other activities for the day
#       ],
#       "approximate_cost": "1200 EGP"
#     },
#     ... // more days
#   ],
#   "total_approximate_cost": "4210 EGP",
#   "notes": "Any assumptions or additional notes."
# }

# Please ensure the output is valid JSON with no extra text.
# """
# )

In [28]:
json_schema = {
    "title": "TravelItinerary",
    "description": "A structured travel itinerary for the user.",
    "type": "object",
    "properties": {
        "days": {
            "type": "array",
            "description": "List of days with planned activities.",
            "items": {
                "type": "object",
                "properties": {
                    "day": {"type": "string", "description": "Theme of the Day or Day label, e.g., 'Day 1'"},
                    "activities": {
                        "type": "array",
                        "description": "Activities planned for the day.",
                        "items": {
                            "type": "object",
                            "properties": {
                                "time": {"type": "string", "description": "Time of the activity"},
                                "activity": {"type": "string", "description": "Name of the activity"},
                                "location": {"type": "string", "description": "Location name"},
                                "price_range": {"type": "string", "description": "Price range or cost"},
                            },
                            "required": ["time", "activity", "location"]
                        }
                    },
                    "approximate_cost": {"type": "string", "description": "Total cost for the day"}
                },
                "required": ["day", "activities", "approximate_cost"]
            }
        },
        "total_approximate_cost": {
            "type": "string",
            "description": "Total cost for the trip"
        },
        "notes": {
            "type": "string",
            "description": "Any additional notes or assumptions"
        }
    },
    "required": ["days", "total_approximate_cost"]
}

structured_llm = llm_model.with_structured_output(json_schema, include_raw=True)



In [17]:
prompt_template = PromptTemplate(
    input_variables=["context", "user_query", "favorite_places", "visitor_type", "num_days", "budget"],
    template="""You are a helpful travel planner AI.
Use the context below, which contains information about ticket prices, place descriptions, restaurant details, and art gallery information.

Context:
{context}

User Query:
{user_query}

Additional Preferences:
- Favorite types of places: {favorite_places}
- Visitor type: {visitor_type} (e.g., Egyptian, Egyptian student, Foreign, or foreign student)
- Number of travel days: {num_days}
- Overall budget for all days: {budget} EGP
- Exclude hotels from the plan.
- Ensure that the itinerary includes at least 3 meals per day.

Based on the above, return a detailed {num_days}-day travel itinerary with approximate costs and suggestions. If some details are missing, make reasonable assumptions and indicate them.
"""
)


In [18]:

def generate_travel_plan(user_query, favorite_places, visitor_type, num_days, budget):
    # 1. Retrieve relevant docs
    docs = retriever.get_relevant_documents(user_query)
    context_text = "\n".join([doc.page_content for doc in docs])
    # 2. Build the final prompt with the additional variables
    prompt = prompt_template.format(
        context=context_text,
        user_query=user_query,
        favorite_places=favorite_places,
        visitor_type=visitor_type,
        num_days=num_days,
        budget=budget
    )
    # 3. Call the LLM
    response = structured_llm.invoke(prompt)
    return response


In [19]:
user_query = "Plan a 3-day trip in Luxor with visits to cultural sites, art galleries, and dining(restaurents) options."
favorite_places = "Cultural sites, historical landmarks, art galleries"
visitor_type = "Foreign"  # or "Egyptian", "Egyptian student", "foreign student"
num_days = "3"
budget = "5000"  # Overall budget in EGP



In [32]:
travel_plan = generate_travel_plan(user_query, favorite_places, visitor_type, num_days, budget)
print(travel_plan)


{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5fw3', 'function': {'arguments': '{\n    "days": [\n        {\n            "day": "Day 1: East Bank Exploration",\n            "activities": [\n                {\n                    "activity": "Breakfast at AHLLAN Restaurant",\n                    "location": "AHLLAN Restaurant",\n                    "price_range": "E£100–200",\n                    "time": "8:00 AM"\n                },\n                {\n                    "activity": "Visit Luxor Temple",\n                    "location": "Luxor Temple",\n                    "price_range": "Foreign Ticket: 500 EGP",\n                    "time": "9:00 AM"\n                },\n                {\n                    "activity": "Lunch at Bab Sharq Luxor",\n                    "location": "Bab Sharq Luxor",\n                    "price_range": "E£100–200",\n                    "time": "12:30 PM"\n                },\n                {\n                    "activ

In [21]:
travel_plan['raw']

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_07dx', 'function': {'arguments': '{"days":[{"day":"Day 1","activities":[{"activity":"Visit Luxor Museum","location":"Kornish Al Nile, Luxor City, Luxor, Luxor Governorate 1362503, Egypt","time":"9:00 AM","price_range":"Foreign Ticket: 400"}],"approximate_cost":"E£ 400"},{"day":"Day 2","activities":[{"activity":"Explore Karnak Temples","location":"Luxor, Luxor Governorate","time":"6:00 AM","price_range":"Foreign Ticket: 600"}],"approximate_cost":"E£ 600"},{"day":"Day 3","activities":[{"activity":"Visit Luxor Art Gallery","location":"Memnon Street, Al Bairat, Al Qarna, Luxor Governorate 1341472, Egypt","time":"10:00 AM","price_range":"Foreign Ticket: 50"}],"approximate_cost":"E£ 50"}],"total_approximate_cost":"E£ 1050"}', 'name': 'TravelItinerary'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 4086, 'total_tokens': 4304, 'completion_time': 0.792727273, 'prompt_time

In [22]:
# travel_plan_qa = qa_chain.invoke(user_query)
# print(travel_plan_qa)